# NYC Airbnb Price Prediction - Deep Learning model training - HELP VERSION

Use dataset published by Kaggle - https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data - to train a simple deep learning model to predict prices for Airbnb properties.

This notebook contains the code to train the model from the dataset prepared in the airbnb_data_preparation notebook. 


# Links to key parts of the notebook <a name='linkanchor' />
<a href=#ingestdash>Ingest data</a>

<a href=#buildpipe>Build pipeline</a>

<a href=#modelfit>Define and fit model</a>

<a href=#reload>Reload saved model and weights</a>

<a href=#confusionmatrix>Confusion matrix</a>

# Common imports and global variable definitions

In [2]:
# common imports
import zipfile
import pandas as pd
import numpy as np
import time
import seaborn as sns
# import datetime, timedelta
import datetime
import pydotplus
from datetime import datetime, timedelta
from datetime import date
from dateutil import relativedelta
from io import StringIO
import pandas as pd
import pickle
from pickle import dump
from pickle import load
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
# DSX code to import uploaded documents
from io import StringIO
import requests
import json
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import os
import yaml
import math
import sys
from subprocess import check_output
from IPython.display import display
#model libraries
from tensorflow.keras.metrics import Accuracy, Recall, Precision
from tensorflow.keras.layers import Input, Dropout, Dense, BatchNormalization, Activation, concatenate, GRU, Embedding, Flatten, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
#from tf.keras.layers.normalization import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
# from tensorflow.keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from tensorflow.python.keras.callbacks import TensorBoard
#import datetime
#from datetime import date
from sklearn import metrics
# import pipeline libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator
from custom_classes import encode_categorical
from custom_classes import prep_for_keras_input
from custom_classes import fill_empty
from custom_classes import encode_text


In [3]:
import tensorflow as tf
tf.__version__ 

'2.0.0'

In [4]:
# load config file
'''
current_path = os.getcwd()
print("current directory is: "+current_path)

path_to_yaml = os.path.join(current_path, 'model_training_config.yml')
print("path_to_yaml "+path_to_yaml)
try:
    with open (path_to_yaml, 'r') as c_file:
        config = yaml.safe_load(c_file)
except Exception as e:
    print('Error reading the config file')
'''


'\ncurrent_path = os.getcwd()\nprint("current directory is: "+current_path)\n\npath_to_yaml = os.path.join(current_path, \'model_training_config.yml\')\nprint("path_to_yaml "+path_to_yaml)\ntry:\n    with open (path_to_yaml, \'r\') as c_file:\n        config = yaml.safe_load(c_file)\nexcept Exception as e:\n    print(\'Error reading the config file\')\n'

In [5]:
# load parameters
'''
repeatable_run = config['test_parms']['repeatable_run']
# fix seeds to get identical results on mulitiple runs
if repeatable_run:
    from numpy.random import seed
    seed(4)
    tf.random.set_seed(7)


testproportion = config['test_parms']['testproportion'] # proportion of data reserved for test set
trainproportion = config['test_parms']['trainproportion'] # proportion of non-test data dedicated to training (vs. validation)
get_test_train_acc = config['test_parms']['get_test_train_acc']
verboseout = config['general']['verboseout']
includetext = config['general']['includetext'] # switch to determine whether text columns are included in the model
save_model_plot = config['general']['save_model_plot'] # switch to determine whether to generate plot with plot_model
tensorboard_callback = config['general']['tensorboard_callback'] # switch to determine if tensorboard callback defined

presaved = config['general']['presaved']
savemodel = config['general']['savemodel']
picklemodel = config['general']['picklemodel']
hctextmax = config['general']['hctextmax']
maxwords = config['general']['maxwords']
textmax = config['general']['textmax']

targetthresh = config['general']['targetthresh']
targetcontinuous = config['general']['targetcontinuous']
target_col = config['general']['target_col']

#time of day thresholds
time_of_day = {'overnight':{'start':0,'end':5},'morning_rush':{'start':5,'end':10},
              'midday':{'start':10,'end':15},'aft_rush':{'start':15,'end':19},'evening':{'start':19,'end':24}}



emptythresh = config['general']['emptythresh']
zero_weight = config['general']['zero_weight']
one_weight = config['general']['one_weight']
one_weight_offset = config['general']['one_weight_offset']
patience_threshold = config['general']['patience_threshold']


# modifier for saved model elements
modifier = config['general']['modifier']

# control whether training controlled by early stop
early_stop = True

# default hyperparameter values
learning_rate = config['hyperparameters']['learning_rate']
dropout_rate = config['hyperparameters']['dropout_rate']
l2_lambda = config['hyperparameters']['l2_lambda']
loss_func = config['hyperparameters']['loss_func']
output_activation = config['hyperparameters']['output_activation']
batch_size = config['hyperparameters']['batch_size']
epochs = config['hyperparameters']['epochs']

# date values
date_today = datetime.now()
print("date today",date_today)

# pickled original dataset and post-preprocessing dataset
pickled_data_file = config['general']['pickled_data_file']
pickled_dataframe = config['general']['pickled_dataframe']

# experiment parameter

current_experiment = config['test_parms']['current_experiment']

# load lists of column categories
collist = config['categorical']
textcols = config['text']
continuouscols = config['continuous']
excludefromcolist = config['excluded']
'''

'\nrepeatable_run = config[\'test_parms\'][\'repeatable_run\']\n# fix seeds to get identical results on mulitiple runs\nif repeatable_run:\n    from numpy.random import seed\n    seed(4)\n    tf.random.set_seed(7)\n\n\ntestproportion = config[\'test_parms\'][\'testproportion\'] # proportion of data reserved for test set\ntrainproportion = config[\'test_parms\'][\'trainproportion\'] # proportion of non-test data dedicated to training (vs. validation)\nget_test_train_acc = config[\'test_parms\'][\'get_test_train_acc\']\nverboseout = config[\'general\'][\'verboseout\']\nincludetext = config[\'general\'][\'includetext\'] # switch to determine whether text columns are included in the model\nsave_model_plot = config[\'general\'][\'save_model_plot\'] # switch to determine whether to generate plot with plot_model\ntensorboard_callback = config[\'general\'][\'tensorboard_callback\'] # switch to determine if tensorboard callback defined\n\npresaved = config[\'general\'][\'presaved\']\nsavemodel 

In [6]:
# get the paths required

def get_path():
    '''get the path for data files

    Returns:
        path: path for data files
    '''


def get_pipeline_path():
    '''get the path for data files
    
    Returns:
        path: path for pipeline files
    '''


def get_model_path():
    '''get the path for data files
    
    Returns:
        path: path for model files
    '''


In [7]:
def set_experiment_parameters(experiment_number, count_no_delay, count_delay):
    ''' set the appropriate parameters for the experiment 
    Args:
        experiment_number: filename containing config parameters
        count_no_delay: count of negative outcomes in the dataset
        count_delay: count of positive outcomes in the dataset

    Returns:
        early_stop: whether the experiment includes an early stop callback
        one_weight: weight applied to positive outcomes
        epochs: number of epochs in the experiment
        es_monitor: performance measurement tracked in callbacks
        es_mod: direction of performance being tracked in callbacks
    
    '''


# Ingest data and create refactored dataframe <a name='ingestdash' />
- Ingest data for route information and delay information
- Create refactored dataframe with one row per route / direction / timeslot combination


<a href=#linkanchor>Back to link list</a>

In [8]:
def ingest_data(path):
    '''load list of valid routes and directions into dataframe
    Args:
        path: path for data files
    
    Returns:
        merged_data: dataframe loaded from pickle file
    '''


In [9]:
def prep_merged_data(merged_data,target_col):
    '''add derived columns to merged_data dataframe
    Args:
        merged_data: input dataframe
        target_col: column that is the target
    
    Returns:
        merged_data: dataframe with derived columns added
    '''


# Master Prep Cell
Contains calls to functions to load data, prep input dataframes, and create refactored dataframe

In [10]:
# master calls
# get the path for data files

'''
path = get_path()
print("path is",path)
# load route direction and delay data datframes
merged_data = ingest_data(path)
merged_data = prep_merged_data(merged_data,target_col)
print("shape of pre refactored dataset", merged_data.shape)
#merged_data['year'].value_counts()
#merged_data.groupby(['Route','Direction']).size().reset_index().rename(columns={0:'count'}).tail(50)
# create refactored dataframe with one row for each route / direction / timeslot combination
print("shape of refactored dataset", merged_data.shape)
count_no_delay = merged_data[merged_data['target']==0].shape[0]
count_delay = merged_data[merged_data['target']==1].shape[0]
print("count under mean ",count_no_delay)
print("count over mean ",count_delay)
# define parameters for the current experiment
experiment_number = current_experiment
early_stop, one_weight, epochs,es_monitor,es_mode = set_experiment_parameters(experiment_number, count_no_delay, count_delay)
print("early_stop is ",early_stop)
print("one_weight is ",one_weight)
print("epochs is ",epochs)
print("es_monitor is ",es_monitor)
print("es_mode is ",es_mode)
'''

'\npath = get_path()\nprint("path is",path)\n# load route direction and delay data datframes\nmerged_data = ingest_data(path)\nmerged_data = prep_merged_data(merged_data,target_col)\nprint("shape of pre refactored dataset", merged_data.shape)\n#merged_data[\'year\'].value_counts()\n#merged_data.groupby([\'Route\',\'Direction\']).size().reset_index().rename(columns={0:\'count\'}).tail(50)\n# create refactored dataframe with one row for each route / direction / timeslot combination\nprint("shape of refactored dataset", merged_data.shape)\ncount_no_delay = merged_data[merged_data[\'target\']==0].shape[0]\ncount_delay = merged_data[merged_data[\'target\']==1].shape[0]\nprint("count under mean ",count_no_delay)\nprint("count over mean ",count_delay)\n# define parameters for the current experiment\nexperiment_number = current_experiment\nearly_stop, one_weight, epochs,es_monitor,es_mode = set_experiment_parameters(experiment_number, count_no_delay, count_delay)\nprint("early_stop is ",early_

# Define training, validation, and test subsets of the dataset

In [11]:
def get_train_validation_test(dataset):
    '''get training and test data set
    Args:
        dataset: input dataframe
    
    Returns:
        dtrain: training subset of dataset
        dvalid: validation subset of dataset
        dtest: test subset of dataset
    '''




# Build Pipeline <a name='buildpipe' />

Create pipeline objects to perform final data preparation steps for training and inference.

Note that cleanup on the training dataset is completed upstream in the [data cleanup notebook](https://github.com/ryanmark1867/end_to_end_deep_learning_liveproject/blob/master/notebooks/data_cleanup.ipynb). 
- The pipelines only accomplish the subset of preparation that is required for both training and inference
- Because the scoring data coming in for inference is forced by the web deployment to avoid the invalid values that the data cleanup notebook deals with, the pipelines don't have to deal with those problems.

<a href=#linkanchor>Back to link list</a>

In [12]:
# master block to invoke pipeline

'''

# build fully qualified names for the files for saving the pipelines
pipeline_path = get_pipeline_path()
pipeline1_file_name = os.path.join(pipeline_path,'sc_delay_pipleline'+modifier+'.pkl')
pipeline2_file_name = os.path.join(pipeline_path,'sc_delay_pipleline_keras_prep'+modifier+'.pkl')

# define column lists:
# collist,continuouscols,textcols = def_col_lists()

# create objects of the pipeline classes
fe = fill_empty()
ec = encode_categorical()
pk = prep_for_keras_input()
pk_valid = prep_for_keras_input()
pk_test = prep_for_keras_input()

# need to implement the pipeline in two parts:
# 1. fill empty + encode categoricals
# 2. prep for Keras
# because part 1 needs to be applied to the entire dataset and part 2 to the individual train, validate, and test sets


sc_delay_pipeline = Pipeline([('fill_empty',fe),('encode_categorical',ec)])
# need to have distinct pipeline objects for each subset of the dataset: train, validated and test
sc_delay_pipeline_keras_prep = Pipeline([('prep_for_keras',pk)])
sc_delay_pipeline_keras_prep_valid = Pipeline([('prep_for_keras',pk_valid)])
sc_delay_pipeline_keras_prep_test = Pipeline([('prep_for_keras',pk_test)])



# provide the value for each parameter of each of the pipeline classes

sc_delay_pipeline.set_params(fill_empty__collist = collist, fill_empty__continuouscols = continuouscols,
                            fill_empty__textcols = textcols,encode_categorical__col_list = collist)
sc_delay_pipeline_keras_prep.set_params(prep_for_keras__collist = collist,
                            prep_for_keras__continuouscols = continuouscols,
                            prep_for_keras__textcols = textcols)
sc_delay_pipeline_keras_prep_valid.set_params(prep_for_keras__collist = collist,
                            prep_for_keras__continuouscols = continuouscols,
                            prep_for_keras__textcols = textcols)
sc_delay_pipeline_keras_prep_test.set_params(prep_for_keras__collist = collist,
                            prep_for_keras__continuouscols = continuouscols,
                            prep_for_keras__textcols = textcols)

# fit the input dataset to the pipeline

# first fit the first segment of pipeline on the whole dataset
X = sc_delay_pipeline.fit_transform(merged_data)
max_dict = ec.max_dict
# then split dataset
dump(sc_delay_pipeline, open(pipeline1_file_name,'wb'))
dump(sc_delay_pipeline_keras_prep, open(pipeline2_file_name,'wb'))
dtrain, dvalid, test = get_train_validation_test(X)
# then apply second portion of pipeline to each subset
# need to have a distinct object for each to prevent first object impacting others

X_train_list = sc_delay_pipeline_keras_prep.fit_transform(dtrain)
X_valid_list = sc_delay_pipeline_keras_prep_valid.fit_transform(dvalid)
X_test_list = sc_delay_pipeline_keras_prep_test.fit_transform(test)

print("keras variables defined")
print("X_train_list",X_train_list)
'''


'\n\n# build fully qualified names for the files for saving the pipelines\npipeline_path = get_pipeline_path()\npipeline1_file_name = os.path.join(pipeline_path,\'sc_delay_pipleline\'+modifier+\'.pkl\')\npipeline2_file_name = os.path.join(pipeline_path,\'sc_delay_pipleline_keras_prep\'+modifier+\'.pkl\')\n\n# define column lists:\n# collist,continuouscols,textcols = def_col_lists()\n\n# create objects of the pipeline classes\nfe = fill_empty()\nec = encode_categorical()\npk = prep_for_keras_input()\npk_valid = prep_for_keras_input()\npk_test = prep_for_keras_input()\n\n# need to implement the pipeline in two parts:\n# 1. fill empty + encode categoricals\n# 2. prep for Keras\n# because part 1 needs to be applied to the entire dataset and part 2 to the individual train, validate, and test sets\n\n\nsc_delay_pipeline = Pipeline([(\'fill_empty\',fe),(\'encode_categorical\',ec)])\n# need to have distinct pipeline objects for each subset of the dataset: train, validated and test\nsc_delay_pi

# Define and fit model <a name='modelfit' />

- define the architecture of the model - the layers determined by the type (categorical, continuous, or text) of each column
- apply callbacks
- fit the model

<a href=#linkanchor>Back to link list</a>

In [13]:
# define model in Keras

def get_model():
    ''' define Keras model by specifying layers by column type
    
    Returns:
        model: Keras model with the layers specified by the structure of the dataset

    '''


In [14]:
# save model plot
'''
if save_model_plot:
    model_plot_file = "model_plot"+modifier+".png"
    model_plot_path = os.path.join(get_path(),model_plot_file)
    print("model plot path: ",model_plot_path)
    plot_model(model, to_file=model_plot_path)
'''

'\nif save_model_plot:\n    model_plot_file = "model_plot"+modifier+".png"\n    model_plot_path = os.path.join(get_path(),model_plot_file)\n    print("model plot path: ",model_plot_path)\n    plot_model(model, to_file=model_plot_path)\n'

In [15]:
# set up early stopping
def set_early_stop(es_monitor, es_mode):
    ''' given monitoring parameter es_monitor and mode es_mode, define early stopping callback, save model callback, and 
    TensorBoard callback
    
    Args:
        es_monitor: the performance parameter to monitor in the callback
        es_mode: the extremity (max or min) to optimize towards
        
    Returns:
        callback_list: list of callback objects
        save_model_path: fully qualified filename to save optimal model to  
    
    
    '''


In [16]:
%%time
# master block to define callbacks & model and fit model 
'''
print("text cols",textcols)
print("dropout ",dropout_rate)
print("L2 lambda ",l2_lambda)
print("batch size ",batch_size)
print("epochs",epochs)
print("learning_rate",learning_rate)
print("loss function",loss_func)
print("output activation function",output_activation)
print("patience_threshold is ",patience_threshold)
print("experiment number is ",experiment_number)
print("early stop is",early_stop)
# get definitions of callbacks
callback_list, save_model_path = set_early_stop(es_monitor, es_mode)
# define model
model = get_model()
# if callbacks are specified 
if early_stop:
       modelfit = model.fit(X_train_list, dtrain.target, epochs=epochs, batch_size=batch_size
        , validation_data=(X_valid_list, dvalid.target), class_weight = {0 : zero_weight, 1: one_weight}, verbose=1,callbacks=callback_list)
else:
    modelfit = model.fit(X_train_list, dtrain.target, epochs=epochs, batch_size=batch_size
         , validation_data=(X_valid_list, dvalid.target), class_weight = {0 : zero_weight, 1: one_weight}, verbose=1)
'''

Wall time: 0 ns


'\nprint("text cols",textcols)\nprint("dropout ",dropout_rate)\nprint("L2 lambda ",l2_lambda)\nprint("batch size ",batch_size)\nprint("epochs",epochs)\nprint("learning_rate",learning_rate)\nprint("loss function",loss_func)\nprint("output activation function",output_activation)\nprint("patience_threshold is ",patience_threshold)\nprint("experiment number is ",experiment_number)\nprint("early stop is",early_stop)\n# get definitions of callbacks\ncallback_list, save_model_path = set_early_stop(es_monitor, es_mode)\n# define model\nmodel = get_model()\n# if callbacks are specified \nif early_stop:\n       modelfit = model.fit(X_train_list, dtrain.target, epochs=epochs, batch_size=batch_size\n        , validation_data=(X_valid_list, dvalid.target), class_weight = {0 : zero_weight, 1: one_weight}, verbose=1,callbacks=callback_list)\nelse:\n    modelfit = model.fit(X_train_list, dtrain.target, epochs=epochs, batch_size=batch_size\n         , validation_data=(X_valid_list, dvalid.target), clas

In [17]:
# block to save model elements explicitly if not saved as part of early_stop
'''
if early_stop == False:
    model_json = model.to_json()
    model_path = get_model_path()
    with open(os.path.join(model_path,'model'+modifier+'.json'), "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(os.path.join(model_path,'scweights'+modifier+'.h5'))
    save_model_path = os.path.join(model_path,'scmodel'+modifier+'.h5')
    model.save(save_model_path,save_format='h5')
    # no early stop, so make current model saved_model
    saved_model = model
    print("Saved model, weights to disk")
'''

'\nif early_stop == False:\n    model_json = model.to_json()\n    model_path = get_model_path()\n    with open(os.path.join(model_path,\'model\'+modifier+\'.json\'), "w") as json_file:\n        json_file.write(model_json)\n    # serialize weights to HDF5\n    model.save_weights(os.path.join(model_path,\'scweights\'+modifier+\'.h5\'))\n    save_model_path = os.path.join(model_path,\'scmodel\'+modifier+\'.h5\')\n    model.save(save_model_path,save_format=\'h5\')\n    # no early stop, so make current model saved_model\n    saved_model = model\n    print("Saved model, weights to disk")\n'

In [18]:
# block to load saved model and get test and train accuracy
'''
saved_model = load_model(save_model_path)
print("metrics names ",saved_model.metrics_names)
# getting test and train accuracy is time consuming - check to see if experiment requires it
if get_test_train_acc:
    if len(saved_model.metrics_names) == 2:
        # saved_model.evaluate returns ['loss', 'acc']
        _, train_acc = saved_model.evaluate(X_train_list, dtrain.target, verbose=0)
        _, test_acc = saved_model.evaluate(X_test_list, test.target, verbose=0)
    else:
        # saved_model.evaluate returns ['loss', 'accuracy', 'accuracy_1']
        _, train_acc,_ = saved_model.evaluate(X_train_list, dtrain.target, verbose=0)
        _, test_acc,_ = saved_model.evaluate(X_test_list, test.target, verbose=0)
    print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
'''



'\nsaved_model = load_model(save_model_path)\nprint("metrics names ",saved_model.metrics_names)\n# getting test and train accuracy is time consuming - check to see if experiment requires it\nif get_test_train_acc:\n    if len(saved_model.metrics_names) == 2:\n        # saved_model.evaluate returns [\'loss\', \'acc\']\n        _, train_acc = saved_model.evaluate(X_train_list, dtrain.target, verbose=0)\n        _, test_acc = saved_model.evaluate(X_test_list, test.target, verbose=0)\n    else:\n        # saved_model.evaluate returns [\'loss\', \'accuracy\', \'accuracy_1\']\n        _, train_acc,_ = saved_model.evaluate(X_train_list, dtrain.target, verbose=0)\n        _, test_acc,_ = saved_model.evaluate(X_test_list, test.target, verbose=0)\n    print(\'Train: %.3f, Test: %.3f\' % (train_acc, test_acc))\n'

# Predictions and renderings

In [19]:
# predictions on training set
'''
preds = saved_model.predict(X_test_list, batch_size=batch_size)
'''

'\npreds = saved_model.predict(X_test_list, batch_size=batch_size)\n'

In [20]:
'''
test["predict"] = preds
test.predict[:5]
if verboseout:
    test.predict.hist()
'''

'\ntest["predict"] = preds\ntest.predict[:5]\nif verboseout:\n    test.predict.hist()\n'

In [21]:
# get rounded predictions
'''
test["predround"] = preds.round().astype(int)
test.predround[:5]
'''

'\ntest["predround"] = preds.round().astype(int)\ntest.predround[:5]\n'

In [22]:
# predict values for validation X values
# X_valid, dvalid.target
'''
predval = model.predict(X_valid_list, batch_size=batch_size)
dvalid["predround"] = predval.round().astype(int)
dvalid["predict"] = predval
#print(type(deltaval))
#print(len(deltaval))
dvalid.predict[:5]
'''




'\npredval = model.predict(X_valid_list, batch_size=batch_size)\ndvalid["predround"] = predval.round().astype(int)\ndvalid["predict"] = predval\n#print(type(deltaval))\n#print(len(deltaval))\ndvalid.predict[:5]\n'

In [23]:
# hand calculation of proportion correct guesses in validation set
'''
dvalid["deltaval"] = abs(dvalid.target - dvalid.predround)
print(dvalid["deltaval"][:10])
print(dvalid["deltaval"].sum())
'''


'\ndvalid["deltaval"] = abs(dvalid.target - dvalid.predround)\nprint(dvalid["deltaval"][:10])\nprint(dvalid["deltaval"].sum())\n'

In [24]:
# hand calculation of proportion correct guesses in validation set
'''
test["deltaval"] = abs(test.target - test.predround)
print(test["deltaval"][:10])
print(test["deltaval"].sum())
'''

'\ntest["deltaval"] = abs(test.target - test.predround)\nprint(test["deltaval"][:10])\nprint(test["deltaval"].sum())\n'

In [25]:
# get subset of dataframe with wrong guesses
# k1 = df.loc[(df.Product == p_id)
'''
dvalidwrong = dvalid.loc[(dvalid.deltaval == 1)]
dvalidright = dvalid.loc[(dvalid.deltaval == 0)]
dvalidwrong.head(20)
'''

'\ndvalidwrong = dvalid.loc[(dvalid.deltaval == 1)]\ndvalidright = dvalid.loc[(dvalid.deltaval == 0)]\ndvalidwrong.head(20)\n'

In [26]:
# get subset of dataframe with wrong guesses
'''
testwrong = test.loc[(test.deltaval == 1)]
testright = test.loc[(test.deltaval == 0)]
testwrong.head(20)
'''

'\ntestwrong = test.loc[(test.deltaval == 1)]\ntestright = test.loc[(test.deltaval == 0)]\ntestwrong.head(20)\n'

In [27]:
# get delta between predictions on training set and actual training target values
# hand calculate accuracy on training set as ratio of (total training samples - wrong training predictions)/total training samples
'''
deltatr = abs(dvalid.target[:100000] - dvalid.predround[:100000])
deltatr[:50]
print(deltatr.sum())
print("percentage correct validate")
print((len(deltatr) - deltatr.sum())/len(deltatr))
'''

'\ndeltatr = abs(dvalid.target[:100000] - dvalid.predround[:100000])\ndeltatr[:50]\nprint(deltatr.sum())\nprint("percentage correct validate")\nprint((len(deltatr) - deltatr.sum())/len(deltatr))\n'

In [28]:
# chart accuracy and loss for train and validation sets
'''
print(modelfit.history.keys())
#  acc
#plt.plot(modelfit.history['accuracy'])
plt.plot(modelfit.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
#plt.legend(['validation'], loc='upper left')
plt.show()
# Loss
plt.plot(modelfit.history['loss'])
plt.plot(modelfit.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
'''

"\nprint(modelfit.history.keys())\n#  acc\n#plt.plot(modelfit.history['accuracy'])\nplt.plot(modelfit.history['val_accuracy'])\nplt.title('model accuracy')\nplt.ylabel('accuracy')\nplt.xlabel('epoch')\n# plt.legend(['train', 'validation'], loc='upper left')\n#plt.legend(['validation'], loc='upper left')\nplt.show()\n# Loss\nplt.plot(modelfit.history['loss'])\nplt.plot(modelfit.history['val_loss'])\nplt.title('model loss')\nplt.ylabel('loss')\nplt.xlabel('epoch')\nplt.legend(['train', 'validation'], loc='upper left')\nplt.show()\n"

# Confusion matrix <a name='confusionmatrix' />
<a href=#linkanchor>Back to link list</a>

In [29]:
'''

cfmap=metrics.confusion_matrix(y_true=test['target'],  # True labels
                         y_pred=test["predround"])

label = ["0", "1"]
sns.heatmap(cfmap, annot = True, xticklabels = label, yticklabels = label)
plt.xlabel("Prediction")
plt.title("Confusion Matrix for Airbnb NYC price prediction (weighted)")
plt.show()
'''

'\n\ncfmap=metrics.confusion_matrix(y_true=test[\'target\'],  # True labels\n                         y_pred=test["predround"])\n\nlabel = ["0", "1"]\nsns.heatmap(cfmap, annot = True, xticklabels = label, yticklabels = label)\nplt.xlabel("Prediction")\nplt.title("Confusion Matrix for Airbnb NYC price prediction (weighted)")\nplt.show()\n'

# Kaggle submission that was used as input for this notebook
https://www.kaggle.com/knowledgegrappler/a-simple-nn-solution-with-keras-0-48611-pl

# Summary
This notebook shows methods for dealing with structured data in the context of a neural network.

# Author

Mark Ryan is a manager at Intact Insurance.